In [20]:
import pandas as pd
import google.generativeai as genai
import os
import re
from configs.api_keys import GEMINI_API_KEY

# metrics 
def edit_distance_similarity(str1, str2):
    m, n = len(str1), len(str2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j  # If str1 is empty, insert all characters of str2
            elif j == 0:
                dp[i][j] = i  # If str2 is empty, remove all characters of str1
            elif str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # If last characters are the same, ignore them
            else:
                dp[i][j] = 1 + min(dp[i][j - 1],  # Insert
                                   dp[i - 1][j],  # Remove
                                   dp[i - 1][j - 1])  # Replace

    edit_dist = dp[m][n]
    max_len = max(m, n)
    similarity = 1 - (edit_dist / max_len)

    return similarity

def edit_distance(str1, str2):
    m, n = len(str1), len(str2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j  # If str1 is empty, insert all characters of str2
            elif j == 0:
                dp[i][j] = i  # If str2 is empty, remove all characters of str1
            elif str1[i - 1] == str2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]  # If last characters are the same, ignore them
            else:
                dp[i][j] = 1 + min(dp[i][j - 1],  # Insert
                                   dp[i - 1][j],  # Remove
                                   dp[i - 1][j - 1])  # Replace
    return dp[m][n]

def word_error_rate(reference, hypothesis):
    reference_words = reference.split()
    hypothesis_words = hypothesis.split()
    edit_dist = edit_distance(reference_words, hypothesis_words)
    return edit_dist / len(reference_words)


def character_error_rate(reference, hypothesis):
    reference_chars = list(reference)
    hypothesis_chars = list(hypothesis)
    edit_dist = edit_distance(reference_chars, hypothesis_chars)
    return edit_dist / len(reference_chars)


def get_gemini_correction(text, api_key):
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-1.0-pro-latest')
    response = model.generate_content(f"""Please fix this email English text. Use only lowercase letters, commas, and periods. Answer in one line: \n {text}""",
                                      safety_settings={
                                        'HATE': 'BLOCK_NONE',
                                        'HARASSMENT': 'BLOCK_NONE',
                                        'SEXUAL' : 'BLOCK_NONE',
                                        'DANGEROUS' : 'BLOCK_NONE'}
                                        )

   
    return response.text

def process_prediction(csv_file):
    clf_result = pd.read_csv(csv_file)
    chars = clf_result[clf_result['Prob'] > 0.9]['Key prediction'].to_list()

    processed = []

    i = 0
    while i < len(chars):
        char = chars[i]
        if char == 'dot': processed.append('.')
        elif char == 'comma': processed.append(',')
        elif char == 'space': processed.append(' ')
        elif char == 'delete':
            if len(processed):
                processed.pop()
        else:
            processed.append(char)
        i += 1
    return ''.join(processed)

def filter_string(input_string):
    # Keep only lowercase and uppercase letters, dots, and commas
    filtered_string = re.sub(r'[^a-zA-Z., ]', '', input_string)
    return filtered_string

def evaluate(csv_file, ground_truth_path):
    prediction = process_prediction(csv_file)
    print('prediction: ', prediction)
    correction = get_gemini_correction(prediction, GEMINI_API_KEY)
    
    correction = filter_string(correction)
    print('correction: ', correction)

    # Open the file in read mode ('r')
    with open(ground_truth_path, 'r') as file:
        # Read the entire content of the file
        gt = file.read()

    print(edit_distance_similarity(correction, gt))
    print(word_error_rate(gt, correction))

In [21]:
evaluate('stream_results/topview/video_6_clf.csv', 'datasets/video-2/ground_truths/video_6.txt')

prediction:  kknonddeeerrr  aaqqquiiannnccyy kk,,  ii   hhhhoopppee  yooou     aaaarree   ddopn  wweeelll   a..    iww    wwwanntteeeeedd  tto   upppddaattee   yyo   oo  lattesttt  aprroojeecctttlk,,,, whhcchhh   hhhass     bbteeennn  qquuitteee    attthhee   jjjjoonuunnnneee k,,...   tthhee  worrkkk   innnvvllllllvvveeees  aaaa  sseeerrrireen  f  ccoommpeeedxx   ttaaask   ss  k,  eeaa dch wiitthh     its   uupnnniqquuies   aaccchhhhaallllellllleenngea...   oour   kteaamm  kk,,,   iinnncccllludduuueddinngg  kkkeennn    aandd  zzeeeelldddaaa,,,  hhhas  bgbgeeeen   aadddeiiddlkiaggeentloyy   ajwwwokkinnng ttottt   jaakkkelleeee  ktttesee   isssssuueel   rrreeedccceeeelllyyy     kkwweee  faacceedd      ffaasseeerfvvvveeeraall   aqquuuiiirrkkyy    ssiiiatttuuattiiooonnss   atthht   eeeguuiiireeasddd  iimmmmeeedeaaatteee   aaaattttttiin   tthhakaaffulllyy  kk,,  wwee   mmannggeeeeeaddd  ttto  adddeeesws  theemm   iitthh tthhe hheppp   oooof  ooour  innnnnoattiivvee     tteeechnnnniiequuuee 

In [28]:
fixed = 'konder aquaincy, i hope you are doing well. i wanted to update you on our latest project, which has been quite the journey. the work involves a series of complex tasks, each with its unique challenges. our team, including ken and zelda, has been diligently working to tackle the issues we recently faced, overcoming several quirky situations that required immediate action. thankfully, we managed to address them with the help of our innovative techniques and quick adjustments. the process has been rigorous, but we are making significant strides. our project is driven by advanced algorithms and cutting-edge technology. the key aspect is accurate data analysis, sifting through extensive datasets with precise measurements. we have had to navigate tricky scenarios, but we are confident in the direction we are heading. if you have any questions or need more details, please feel free to reach out. we are available for a quick call if that would be more convenient. your feedback and insight are highly valued and will help us ensure the project meets all expectations. thank you for your continued support and understanding. we look forward to sharing more updates with you soon. best regards, jack quimby zimmer.'
with open('datasets/video-2/ground_truths/video_6.txt', 'r') as file:
    # Read the entire content of the file
    gt = file.read()
    print(edit_distance_similarity(gt, fixed))

0.9360655737704918


In [23]:
evaluate(
    'stream_results/topview/video_7_clf.csv', 
    'datasets/video-2/ground_truths/video_7.txt'
)

prediction:  qqdddeerr     tteemmammm,,     ii   hooopppeaayyoo   ll     haaddd  aa   faannstttaaaaaaassstttiicc  e,  ojou   eedccnettt   tteeeemllllyyyeeeee   eee pppppprreeedcciiaaaatttioon   eeeee vvnnntt..  ii  tt   aawassssd   ssttuuuulyy   aa  aplleeewure    ttt   seeeee   deeevveeroonnee    ccoome  ttetthheer aaaa  aaddd  eenujoyy  dtthee    ffeeeeatttssatt  vvittttieo..  yyyooour    aeeehhuussiissaasssmmm    aaandd  tt vvveee   aeeeevvneeerryyyyyy      aammedddeee e   tttheeee veeeett   aaa   reeeaouunnnddiinngg   acccsess ...  wweee    sswwnnntt edd  to  ttaaakk    maw  mmomeeenntt      ttteoo  ttteeeettteeenndd   ooouurr   hearteeeelll t   gggraaddtifuuutuedeee    to   eee hh       oof yyoouure rrdd  wwoorrkkee  dd  deeedddediccaattttn..  eee veenttt     ikkeee    tthheeeee  arre a   sss mmmall   ttloeeenn     oeff   aaappppprreeedcciieatttioonn      ffoe  tthhee  iiiinnceeeddddiidccbbllee   eeefffette oo opuut   kkkkiinnntto   yyoourr   rrollddddeess eeeveyye  ddaqttyyyy... 

In [27]:
fixed = "dear team, i hope you all had a fantastic time, enjoyed the event. it was truly a pleasure to see everyone come together and enjoy the feats at the event. your enthusiasm and the energy made the event a resounding success. we want to take a moment to extend our heartfelt gratitude to each of you for your work and dedication. the event like this is a small token of appreciation for the incredible effort put into your roles every day. as we move forward, let's carry this spirit of camaraderie and teamwork. your contributions are invaluable to our company’s success, and we look forward to achieving even greater milestones together. once again, thank you for being an integral part of our team. here's to continued success and many more memorable moments ahead. best regards, katy"
with open('datasets/video-2/ground_truths/video_7.txt', 'r') as file:
    # Read the entire content of the file
    gt = file.read()
    print(edit_distance_similarity(gt, fixed))

0.8962962962962963


In [24]:
evaluate('stream_results/video_6_clf.csv', 'datasets/video/ground_truths/video_6.txt')

prediction:  deearrr qquuiccanccyy,,,,  i  hhioopeee uuuuyouua     are eddooinngnaweell,...  ffii wwwannntteeedd   etto uhpddaaeee tteeefyoouu  oonn ooouuurraaatttesssat   fpporroooneeccttttt,,, wwhhicchh   hass beeeef      quutteee ttheeefjooarrnnreey...  tthheeeawwworrkkk  finnnvvaolllvveessa a  ssseerreiiseeess aon ccoomppleexx tasaksa eeeaaccchh awwieth  itts aauuniiaueee  cccchaallleeenngees ...   oourr tteeemaam,, inccclluddinnggekkeedvveen   aannd zeellddda,, haaa    avveeenn  ddeilliigeenttllyy lcaawwoorkinngg tottacckkleee  aatthheeeee eisssuess..  rreeeccenntauyy  wwee dfacceedaaa assevveerall  aaquiirkeyy ssittuuaaraaattioonnss  tah frequiirredd  iifmmmmediiaaatee  aaatteeeeddtttioon.. tthhaaannnkaaffullllyyf,,, wwee  maaamaaannaaaaggeeddd to   aaddddddrheem wwiithhh  tttheehhelp   ooff ourr  fiiffnnooaaive tteecchanaaiiaueess aaaandd qqquiccka  aaaaddaauussatammeenttsa... tthee epprroocceeesssa  haassa  eeeen  fquuiiettteeerggooraoouu,, bu aaaeeaaare aaaaaaaknngg  ssaggnnnn

In [25]:
evaluate(
    'stream_results/video_7_clf.csv', 
    'datasets/video/ground_truths/video_7.txt'
)

prediction:  deear tteeaama,,,  ffaiie hoopee uyuyouu  aalll afhhaaadd aa faattsstticf tiaammee oiuurrrrreccceenttt emmpplleeoyyeee aapppeecciaatiioonneevveennett...  iitt cwaaasssttrrully aa plleeaasurre  ttt  sseeeee  eeevveerryyooneee  ccoommeee  togggaeetthherr srtaanndd fjeeennaooyya   ttheee ffeestt aavvvifaattiss.  yyoouuurrrr  eeentthhuusiaassmmma  aaand s poosssiittiivvee  eenneergyy  maadee  tthtthheeeeeeveentt aaa  arressnnadiinnggea  suucccccessa.. wweee  waaaanntteddd  ttoetaakkee aaa  mmeennntt  tttoo seexxteendd oouuurrqqhheeaarrtaaffaeellt gattiitttttuude  attoeeeeeecchh   oooff  yyouuurr aaaardd  wwworrrkk  aaanndd  deedddiiccaatiioonna...  eeedvveenttsllike tpeeseee aareeaaa  sammaalla   tttooekkeen ooouuuf aappppreeeciiattionnn  ffoor ttheeffiincccrrddiblle eefforr  ettuyyouu ppuutt inntto   yyoouuurr rroollee eeevaeeryayyla.... aass mwwweemoooevveeeee foorawwaa,, letssaccaaaryy aaforrawwwaaarddfatthiissasspararritt  off cccaammaarradeeiirie  aaddd tteaamma vawwoorrk